<a href="https://colab.research.google.com/github/GDharan10/Project8_ClassificationModelForFinancialStatements/blob/main/Classification_Model_For_Financial_Statementsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [2]:
import os
import pandas as pd
import re

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data collection**

In [24]:
# Function to clean the text
def clean_text(text):
    # Remove specified substrings and non-alphanumeric characters, except spaces
    cleaned_text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Folder path containing HTML files
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Balance Sheets"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        # Read HTML tables into a list of DataFrame objects
        dfs = pd.read_html(file_path)
        # Assuming the desired table is the first one
        table_df = dfs[0]
        # Extract text from the second column
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, 1] if pd.notnull(cell))
        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)
        # Create a DataFrame with cleaned text and document name
        df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Balance Sheets"})
        # Append the DataFrame to the list
        dfs_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
BalanceSheets_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(BalanceSheets_df)


                                        Extracted_text   document_name
0    Particulars No ASSETS NonCurrent Assets Proper...  Balance Sheets
1                         STANDALONE March Audited III  Balance Sheets
2    GRINDWELL NORTON LIMITED Statement of Standalo...  Balance Sheets
3    ASSETS NonCurrent Assets Property Plant and Eq...  Balance Sheets
4                                  N SRBCCO LLP MUMBAI  Balance Sheets
..                                                 ...             ...
265  AUDITED CONSOLIDATED STATEMENT OF ASSETS AND L...  Balance Sheets
266  Notes to standalone I ml AS financial results ...  Balance Sheets
267  Particulars ASSETS NonCurrent Assets Property ...  Balance Sheets
268  Particulars ASSETS Noncurrent assets Property ...  Balance Sheets
269  Particulars Assets NonCurrent Assets Property ...  Balance Sheets

[270 rows x 2 columns]


In [25]:
BalanceSheets_df.Extracted_text.unique()

array(['Particulars No ASSETS NonCurrent Assets Property Plant and Equipment Capital WorkInProgress Investment Property Goodwill Other Intangible Assets Intangible Assets Under Development Investment in an Associate Financial Assets Investments Loans Others Deferred Tax Assets Net j Other NonCurrent Assets Sub Total NonCurrent Assets Current Assets Inventories Financial Assets Investments Trade Receivables Cash and Cash Equivalents Bank Balance other than above Others Current Tax Assets Net Other Current Assets Sub Total Current Assets TOTAL ASSETS EQUITY AND LIABILITIES EQUITY Equity Share Capital Other Equity Non Controlling Interest Sub Total Equity LIABILITIES NonCurrent Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Provisions Deferred Tax Liabilities Net Other NonCurrent Liabilities Sub Total NonCurrent Liabilities Current Liabilities Financial Liabilities Borrowings Trade Payable Other Financial Liabilities Other Current Liabilities Provis

In [26]:
# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Cash Flow"

# Function to extract text from HTML files and clean it
def extract_and_clean_text(file_path):
    # Read HTML tables into a list of DataFrame objects
    dfs = pd.read_html(file_path)

    # Combine text from all tables into a single string
    extracted_text = ' '.join(' '.join(str(cell) for cell in table_df.iloc[:, 1] if pd.notnull(cell)) for table_df in dfs)

    # Remove unwanted characters and symbols
    cleaned_text = re.sub(r'[^a-zA-Z\s]|[^\w\s]', '', extracted_text)

    # Remove numbers
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# List to store extracted data
data = []

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".html"):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Extract and clean text from the file
        extracted_text = extract_and_clean_text(file_path)

        # Add extracted text and document name to the data list
        data.append({'Extracted_text': extracted_text, 'document_name': "Cash Flow"})

# Create a DataFrame from the extracted data
CashFlow_df = pd.DataFrame(data)

# Display the DataFrame
print(CashFlow_df)

                                       Extracted_text document_name
0                                               As on     Cash Flow
1                                                         Cash Flow
2                                 Year ended st March     Cash Flow
3   DESCRIPTION Cash Inflow Outflow From Operating...     Cash Flow
4              INR in Crores For the year ended March     Cash Flow
5   Particulars Cash Flow from Operating Activitie...     Cash Flow
6                                                         Cash Flow
7   CASH FLOW FROM OPERATING ACTIVITIES Net Profit...     Cash Flow
8   Cash Flow from Operating Activities Net Profit...     Cash Flow
9                                 Year ended December     Cash Flow
10  assets net For Price Waterhouse Chartered Acco...     Cash Flow
11  As at December CASH FLOW FROM FINANCING ACTIVI...     Cash Flow
12              B CASH FLOW FROM INVESTING ACTIVITIES     Cash Flow
13                                Year ended st 

In [27]:
CashFlow_df.Extracted_text.unique()

array(['As on', '', 'Year ended st March',
       'DESCRIPTION Cash Inflow Outflow From Operating Activities Profit Before Tax AddLess Depreciation Interest Income Interest Expense Profit on sale of fixed assets Operating Profit before Working Capital changes Adjustment for Inventories Trade and other Receivable Trade and other Payable Cash Generated From Operations Before Tax Tax Paid Tax Refund Cash Inflow Outflow From Operating Activities Profit Before Tax AddLess Depreciation Interest Income Interest Expense Profit on sale of fixed assets Operating Profit before Working Capital changes Adjustment for Inventories Trade and other Receivable Trade and other Payable Cash Generated From Operations Before Tax Tax Paid Tax Refund Cash Inflow Outflow From Operating Activities Profit Before Tax AddLess Depreciation Interest Income Interest Expense Profit on sale of fixed assets Operating Profit before Working Capital changes Adjustment for Inventories Trade and other Receivable Trade and ot

In [13]:
# Function to clean text
def clean_text(text):
    # Remove specified substrings
    substrings_to_remove = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'IX', 'X', 'XIII', 'XIV', 'XV', 'XVI', 'XVII',
                            'XVIII', 'XIX', 'XX', 'XXI', 'XXII', 'XXIII', 'XXIV', 'XXV', 'XXVI']

    for substr in substrings_to_remove:
        text = re.sub(r'\b{}\b'.format(substr), '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Income Statement"

# Initialize an empty list to store dataframes
dfs = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an HTML file
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the second column
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, 1] if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a new DataFrame
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Income Statement"})

        # Append the new DataFrame to the list
        dfs.append(new_df)

# Concatenate all DataFrames in the list
IncomeStatement_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(IncomeStatement_df)

                                        Extracted_text     document_name
0    Particulars Income Revenue from Operations Oth...  Income Statement
1    Particular Income Revenue from operations Othe...  Income Statement
2    Particulars Income Income from operations refe...  Income Statement
3    QUvfttr muled wined Rrrfi6 2xt L05 OS Ifi HI 0...  Income Statement
4             Audited Consolidated Results Year Ended   Income Statement
..                                                 ...               ...
300  Particulars Particulars Revenue from operation...  Income Statement
301  KPlT Technologies Limited Registered Corporate...  Income Statement
302  Finolex Cables Limited Registered Office Mumba...  Income Statement
303                                            Q4FV18   Income Statement
304  _ _ _ Hi Hn _ member of Registered Office Phon...  Income Statement

[305 rows x 2 columns]


In [14]:
IncomeStatement_df.Extracted_text.unique()

array(['Particulars Income Revenue from Operations Other Income Total Income Expenses Cost of materials consumed Purchases of Stock in Trade Changes in inventories of Finished Goods Work in Progress and Stock in Trade Employees benefits expense Excise Dutv Finance Costs Depreciation and amortisation expense Other expenses Total expenses Profit before Exceptional Item and Tax Exceptional Item Profit before Tax Tax Expenses Current Deferred Total Net Profit after ax Other Comprehensive Income net of tax items that will not be reclassified to Statement of profit and loss in subsequent periods Total Comprehensive Income Paid up Equity Share Capital Face Value Re Other Equity Earnings per Share Basic Diluted ',
       'Particular Income Revenue from operations Other income Total income Expenses Cost of raw materials consumed Purchase of stockintrade Purchase of goods and services of IT division Change in inventories of finished goods work inprogress and stockintrade Power fuel and other uti

In [28]:
def clean_text(text):

    text = re.sub(r'[^a-zA-Z\s]|(?<!\w)(A|B|C|a|b|c|d|e|f|g|h|i|ii|iii|iv|v|vi)(?!\w)', '', text)

    # Remove lowercase letters if they appear separately
    text = re.sub(r'\b[a-z]\b', '', text)

    # Remove uppercase letters if they appear separately
    text = re.sub(r'\b[A-Z]\b', '', text)

    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    # Remove special characters
    cleaned_text = re.sub(r'[^\w\s]+', '', text)

    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

# Define the folder path
folder_path = "/content/drive/MyDrive/Task/FinacPlus/data/data/Notes"

# Initialize an empty list to store DataFrames
dfs_list = []

# Process each HTML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        # Read HTML file into a DataFrame
        table_df = pd.read_html(os.path.join(folder_path, filename))[0]

        # Extract text from the second column
        extracted_text = ' '.join(str(cell) for cell in table_df.iloc[:, 1] if pd.notnull(cell))

        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)

        # Create a DataFrame with the extracted text and document name
        new_df = pd.DataFrame({'Extracted_text': [cleaned_text], 'document_name': "Notes"})

        # Append the DataFrame to the list
        dfs_list.append(new_df)

# Concatenate all DataFrames in the list into a single DataFrame
Notes_df = pd.concat(dfs_list, ignore_index=True)

# Display the final DataFrame
print(Notes_df)

                                        Extracted_text document_name
0                                 Year ended December          Notes
1                                      As at st March          Notes
2     in Crores Refer Note Financial liabilities me...         Notes
3                                         As at Rs Ps          Notes
4    Particulars Particulars RESERVES AND SURPLUS S...         Notes
..                                                 ...           ...
685  Freehold land Cost Depreciationamortization Im...         Notes
686                              December INR in Lacs          Notes
687                                          December          Notes
688                        LONG TERM NONCURRENT March          Notes
689       Cash and cash equivalents December December          Notes

[690 rows x 2 columns]


In [29]:
Notes_df.Extracted_text.unique()

array(['Year ended December ', 'As at st March ',
       ' in Crores Refer Note Financial liabilities measured at fair value through profit or loss FVTPL Financial liabilities measured at amortised cost ',
       'As at Rs Ps ',
       'Particulars Particulars RESERVES AND SURPLUS Securities Premium Reserve As at the commencement of the year Add Additions during the year Less Utilized during the year Surplus Opening Balance Profit and Loss Account Add Transfer from Profit Loss Account Less Change in accounting policy of Depreciation Less Dividend Less Bonus shares Less Transfer tofrom reserves Reserve specifically represented by earmarked investments Negative balance of reserves and surplus account Total Reserves and Surplus',
       ' ', 'Computer Software ', 'As at Mar ',
       'For the year ended March ',
       'Provision for employee benefits Refer note Total Total Total',
       'Gross Block As at April ',
       ' Details of shareholders holding more than shares in the Company 